In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert the labels to one-hot encoded vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create a convolutional neural network
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the testing data
loss, accuracy = model.evaluate(x_test, y_test)

print(f"Test accuracy: {accuracy}")

Epoch 1/10
1563/1563 [==============================] - 15s 10ms/step - loss: 1.5418 - accuracy: 0.4378 - val_loss: 1.2591 - val_accuracy: 0.5466
Epoch 2/10
1563/1563 [==============================] - 15s 9ms/step - loss: 1.1757 - accuracy: 0.5830 - val_loss: 1.0853 - val_accuracy: 0.6155
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.0269 - accuracy: 0.6390 - val_loss: 0.9922 - val_accuracy: 0.6537
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.9359 - accuracy: 0.6735 - val_loss: 0.9864 - val_accuracy: 0.6589
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8619 - accuracy: 0.6981 - val_loss: 0.9192 - val_accuracy: 0.6827
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8023 - accuracy: 0.7185 - val_loss: 0.9078 - val_accuracy: 0.6831
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7553 - accuracy: 0.7358 - val_loss: 0.8845 - 

In [4]:
import tensorflow as tf

In [6]:
#chech if cuda is available with tensorflow
tf.config.list_physical_devices('GPU')

[]